<a href="https://colab.research.google.com/github/thiagomedina/mvp_40530010063_20240_01/blob/main/MVP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação de Movimentos Percentuais do Bitcoin(BTC)
 O notebook apresenta a resolução de um problema de classificação baseado nos movimentos percentuais do Bitcoin. O objetivo é prever se a variação percentual diária será maior que um limite pré-definido.



**Etapas abordadas:**
1. Carga dos dados
2. Pré-processamento
3. Treinamento dos modelos (KNN, Decision Tree, Naive Bayes e SVM)
4. Otimização de hiperparâmetros
5. Avaliação e comparação dos resultados
6. Exportação do modelo final

## Carga dos Dados

a lib `yfinance` é utilizada para carregar os dados históricos. No intervalo de 5 anos.

Em seguida, são criados alguns indicadores, como as médias móveis de 10 e 20 dias, que ajudam a suavizar a volatilidade e identificar tendências de longo prazo, o índice de força relativa para capturar a força do movimento de preço.

O objetivo é prever se o preço do Bitcoin irá aumentar mais de 1% no próximo dia, com base em dados históricos e alguns indicadores para analisar tendencia. O resuldado final será:

1 == Quando a variação percentual do preço de fechamento em relação ao dia anterior é superior a 1%

0 == Quando a variação é igual ou inferior a 1%






In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import joblib
from collections import Counter
from imblearn.over_sampling import SMOTE

data = yf.download('BTC-USD', period='5y', interval='1d')

data['MA10'] = data['Close'].rolling(window=10).mean()
data['MA20'] = data['Close'].rolling(window=20).mean()

delta = data['Close'].diff(1)
delta = delta.dropna()
up = delta.copy()
down = delta.copy()
up[up < 0] = 0
down[down > 0] = 0
time_period = 14
AVG_Gain = up.rolling(window=time_period).mean()
AVG_Loss = abs(down.rolling(window=time_period).mean())
RS = AVG_Gain / AVG_Loss
RSI = 100.0 - (100.0 / (1.0 + RS))
data['RSI'] = RSI

data['Volatility'] = data['Close'].rolling(window=10).std()

data['Price_Change'] = data['Close'].pct_change()
threshold = 0.01
data['Trend'] = np.where(data['Price_Change'] > threshold, 1, 0)
data['Trend'] = data['Trend'].shift(-1)


data = data.dropna()
features = ['Open', 'High', 'Low', 'Volume', 'MA10', 'MA20', 'RSI', 'Volatility']
X = data[features]
y = data['Trend']

[*********************100%***********************]  1 of 1 completed


## Separação entre conjuntos de treino e teste

Foi dividido em dois conjuntos: 80% para treino e 20% para teste. O conjunto de treino é utilizado para treinar, e o conjunto de teste servirá para avaliar a performance em dados não vistos.




In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print('distribuição antes do balanceamento:', Counter(y_train))

sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

print('distribuição depois do balanceamento:', Counter(y_train_res))

distribuição antes do balanceamento: Counter({0.0: 1359, 1.0: 87})
distribuição depois do balanceamento: Counter({0.0: 1359, 1.0: 1359})


## Criação de Pipelines

Pipelines para os modelos KNN, SVM, Árvore de Decisão e Naive Bayes.


In [ ]:
pipelines = {
    'knn': Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier())]),
    'dt': Pipeline([('scaler', StandardScaler()), ('dt', DecisionTreeClassifier())]),
    'nb': Pipeline([('scaler', StandardScaler()), ('nb', GaussianNB())]),
    'svm': Pipeline([('scaler', StandardScaler()), ('svm', SVC())])
}

param_grids = {
    'knn': {'knn__n_neighbors': [3, 5, 7, 9, 11]},
    'dt': {'dt__max_depth': [None, 5, 10, 15], 'dt__min_samples_split': [2, 5, 10]},
    'nb': {},
    'svm': {'svm__C': [0.1, 1, 10, 100], 'svm__kernel': ['linear', 'rbf', 'poly']}
}

best_models = {}
best_scores = {}

## Otimização de Hiperparâmetros com GridSearchCV e cross-validaton
Para cada modelo, é utilizado `GridSearchCV` para otimizar os hiperparâmetros e `cross-validation` para validar os resultados. Os modelos são avaliados com base em métricas como acurácia e F1-score, para balancear a sensibilidade do modelo tanto para acertos quanto para erros.


In [ ]:
for key in pipelines.keys():
    grid = GridSearchCV(pipelines[key], param_grids[key], cv=5, scoring='accuracy')
    grid.fit(X_train_res, y_train_res)
    best_models[key] = grid.best_estimator_
    scores = cross_val_score(grid.best_estimator_, X_train_res, y_train_res, cv=5, scoring='accuracy')
    best_scores[key] = scores.mean()
    print(f"Modelo: {key}, Acuracia media: {scores.mean():.4f}")


Modelo: knn, Acuracia media: 0.8183
Modelo: dt, Acuracia media: 0.8208
Modelo: nb, Acuracia media: 0.5865
Modelo: svm, Acuracia media: 0.7991


## Exportação do Melhor Modelo
# Análise dos Resultados

Após o treinamento, o modelo SVM apresentou o melhor desempenho, a média móvel de 10 dias foi um dos indicadores mais relevantes para o modelo, enquanto o índice de volatilidade teve menos impacto.

O modelo teve dificuldade em prever corretamente mudanças bruscas de tendência,isso pode indicar a necessidade de incorporar mais dados ou de testar novos modelos.



In [ ]:

best_model_key = max(best_scores, key=best_scores.get)
best_model = best_models[best_model_key]
print(f"Melhor modelo: {best_model_key}")

best_model.fit(X_train_res, y_train_res)
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Acuracia do melhor modelo ({best_model_key}) no conjunto de teste: {accuracy:.4f}")
print(classification_report(y_test, y_pred))

joblib.dump(best_model, 'bitcoin_trend_model.pkl')
joblib.dump(X_test, 'x_test.pkl')
joblib.dump(y_test, 'y_test.pkl')

Melhor modelo: dt
Acuracia do melhor modelo (dt) no conjunto de teste: 0.7293
              precision    recall  f1-score   support

         0.0       0.94      0.76      0.84       340
         1.0       0.08      0.32      0.12        22

    accuracy                           0.73       362
   macro avg       0.51      0.54      0.48       362
weighted avg       0.89      0.73      0.80       362



['y_test.pkl']

# Conclusão

Foi desenvolvido um modelo de previsão de tendência do Bitcoin. Os resultados mostraram que o **SVM** foi o mais eficiente.

Alguns passos que podem ajudar a melhorar o modelo no futuro:
- Coletar mais dados históricos para aumentar o tamanho do conjunto de treino.
- Testar modelos mais complexos, como LSTM, que são melhores para séries temporais.

